<a href="https://colab.research.google.com/github/ArtHommage/HommageTools/blob/main/notebooks/comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [1]:
# Cell 1: Create system-level patch
# Create patch file
with open('/usr/local/lib/python3.11/dist-packages/huggingface_hub_patch.py', 'w') as f:
    f.write('''
# Monkey patch huggingface_hub to add cached_download
from functools import wraps
import os
import tempfile
import huggingface_hub

if not hasattr(huggingface_hub, 'cached_download'):
    def cached_download(url, library_name=None, library_version=None, cache_dir=None, **kwargs):
        print(f"Using patched cached_download for: {url}")
        from huggingface_hub import hf_hub_download
        parts = url.split('/')
        if 'huggingface.co' in url:
            try:
                repo_idx = parts.index('huggingface.co') + 2
                repo_id = '/'.join(parts[repo_idx:repo_idx+2])
                filename = parts[-1]
                return hf_hub_download(repo_id=repo_id, filename=filename, cache_dir=cache_dir)
            except:
                pass
        # Fallback
        return url

    # Add to module
    huggingface_hub.cached_download = cached_download
    print("HuggingFace Hub patched with cached_download function")
''')

# Prepend import to existing huggingface_hub/__init__.py
with open('/usr/local/lib/python3.11/dist-packages/huggingface_hub/__init__.py', 'r') as f:
    init_content = f.read()

with open('/usr/local/lib/python3.11/dist-packages/huggingface_hub/__init__.py', 'w') as f:
    f.write('import huggingface_hub_patch\n')
    f.write(init_content)

print("System-level patch applied. Restart the runtime and try again.")

System-level patch applied. Restart the runtime and try again.


In [1]:
# Cell 2: Install Dependencies (Run this cell AFTER restarting the runtime)
# -----------------------------------------------------------------------
print("Installing jedi (for ipython)...")
!pip install "jedi>=0.16"

print("\nInstalling specific NumPy version required by dependencies...")
!pip install "numpy>=2.0.0,<2.1.0"

print("\nInstalling transformers with no-deps...")
!pip install --no-deps transformers==4.31.0

print("\nInstalling additional required dependencies...")
!pip install tokenizers==0.13.3 filelock requests tqdm pyyaml packaging

print("\nInstalling main project packages...")
!pip install \
    transparent_background \
    piexif \
    ultralytics \
    surrealist \
    blend_modes \
    gguf \
    dynamicprompts \
    pygtrans \
    segment_anything \
    dill \
    boto3 \
    fake_useragent \
    redis \
    fal_client \
    replicate \
    psd-tools \
    onnxruntime \
    oidn \
    color-matcher \
    albumentations \
    omegaconf \
    pytorch_lightning \
    open_clip_torch \
    xformers \
    accelerate>=1.6.0 \
    diffusers==0.26.3 \
    scipy>=1.12.0 \
    torchsde>=0.2.6 \
    einops \
    safetensors

!pip install streamdiffusion
!pip install hunyuan-video transformers-stream-generator

Installing jedi (for ipython)...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.6 MB/s eta 0:00:00

Installing specific NumPy version required by dependencies...

Installing transformers with no-deps...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 74.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3

Installing additional required dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following 

In [2]:
#@title Environment Setup


OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUINew"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI from main branch =-
  !git fetch origin
  !git checkout master
  !git reset --hard origin/master
  !git pull origin master

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

!pip install "numpy>=2.0.0,<2.1.0"

Mounting Google Drive...
/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
-= Initial setup ComfyUI =-
fatal: destination path 'ComfyUI' already exists and is not an empty directory.
[Errno 2] No such file or directory: '/content/drive/MyDrive/ComfyUINew'
/content/drive/MyDrive
-= Updating ComfyUI from main branch =-
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not

### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

--2025-04-23 01:01:29--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.4.0/cloudflared-linux-amd64.deb [following]
--2025-04-23 01:01:29--  https://github.com/cloudflare/cloudflared/releases/download/2025.4.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/d7e7703c-c0be-4512-b40f-145c402e03fd?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250423%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250423T005947Z&X-Amz-Expires=300&X-Amz-Signature=dfd18cccd687ea5b26b5d6e6f5d27d1a27a07a8036da5343c5cf84e4ff